In [148]:
import pandas as pd
import numpy as np
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import category_encoders
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
import xgboost as xgb
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from mlxtend.classifier import StackingClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import BaggingClassifier
import matplotlib as plt
from random import randint

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder



print("Setup Complete")

Setup Complete


In [149]:
# Read the file into a variable fifa_data
filepath = "/home/ryan/stark/MachineHack/Food_QUalityA_ParticipantsData/Data_Train.xlsx"
data = pd.read_excel(filepath)


In [150]:
# testdata = pd.read_excel('/home/ryan/stark/MachineHack/Food_QUalityA_ParticipantsData/Data_Test.xlsx')
# data.head(3)

In [151]:
from sklearn.model_selection import train_test_split
y = data.Inspection_Results


# To keep things simple, we'll use only numerical predictors
X = data.drop(['Inspection_Results'], axis=1)

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.65, test_size=0.35,random_state=0)

In [152]:
X_valid.head(2)

,ID,Date,LicenseNo,FacilityID,FacilityName,Type,Street,City,State,LocationID,Reason,SectionViolations,RiskLevel,Geo_Loc
76851,24443412747647,09-07-2018,28225,4702,4480,RESTAURANT,13072,id-11235901,id_1890134,81875.0,CANVASS,2.0,High,locid15302
36203,47254212658879,17-11-2014,743,25793,24635,CAFE/STORE,2087,id-11235901,id_1890134,81882.0,CANVASS,32.0,Medium,locid3211


In [153]:
class LabelEncoderExt(object):
    def __init__(self):
        """
        It differs from LabelEncoder by handling new classes and providing a value for it [Unknown]
        Unknown will be added in fit and transform will take care of new item. It gives unknown class id
        """
        self.label_encoder = LabelEncoder()
        # self.classes_ = self.label_encoder.classes_

    def fit(self, data_list):
        """
        This will fit the encoder for all the unique values and introduce unknown value
        :param data_list: A list of string
        :return: self
        """
        self.label_encoder = self.label_encoder.fit(list(data_list) + ['Unknown'])
        self.classes_ = self.label_encoder.classes_

        return self

    def transform(self, data_list):
        """
        This will transform the data_list to id list where the new values get assigned to Unknown class
        :param data_list:
        :return:
        """
        new_data_list = list(data_list)
        for unique_item in np.unique(data_list):
            if unique_item not in self.label_encoder.classes_:
                new_data_list = ['Unknown' if x==unique_item else x for x in new_data_list]

        return self.label_encoder.transform(new_data_list)
    
    
    
    
    
# from sklearn.tree import DecisionTreeClassifier

# from sklearn.ensemble import BaggingClassifier

# clf1 = DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
#                        max_depth=None, max_features=11, max_leaf_nodes=None,
#                        min_impurity_decrease=0.0, min_impurity_split=None,
#                        min_samples_leaf=8, min_samples_split=2,
#                        min_weight_fraction_leaf=0.0, presort='deprecated',
#                        random_state=None, splitter='best')

# clf2 = BaggingClassifier(base_estimator=clf1, n_estimators=100, max_samples=1.0, max_features=1.0, 
#                          bootstrap=True,
#                          bootstrap_features=False, oob_score=True, warm_start=False,
#                          n_jobs=-1, random_state=None, verbose=0)

# clf3=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
#               colsample_bynode=1, colsample_bytree=0.8, gamma=0.5,
#               learning_rate=0.1, max_delta_step=0, max_depth=5,
#               min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
#               nthread=None, objective='multi:softprob', random_state=0,
#               reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
#               silent=None, subsample=0.6, verbosity=1)  

In [2]:
def dropcol(train,test):
    train['logID'] = np.log(train['ID'])
    test['logID'] = np.log(test['ID'])
    cols_to_drop = ['Date','ID']
    dtrain = train.drop(cols_to_drop,axis =1)
    dtest = test.drop(cols_to_drop,axis =1)
    
    print(dtrain.shape,dtest.shape)

    return dtrain,dtest

def catVar1(data):
    categorical_colsT1 = [cname for cname in data.columns if
                    data[cname].nunique() <=10 and 
                    data[cname].dtype == "object"]
    return categorical_colsT1

def catVar2(data):
    categorical_colsT2 = [cname for cname in data.columns if
                    data[cname].nunique() >10 and 
                    data[cname].dtype == "object"]
    return categorical_colsT2

def NumVar(data) :
    numerical_cols = [cname for cname in data.columns if 
    data[cname].dtype in ['int64', 'float64']]
    return numerical_cols

def imputer(train,test):
    
    from sklearn.impute import SimpleImputer
    
    numerical_cols = NumVar(train)
    my_imputer = SimpleImputer(strategy='most_frequent')
    imputed_train = pd.DataFrame(my_imputer.fit_transform(train))
    imputed_test = pd.DataFrame(my_imputer.transform(test))

    # Imputation removed column names; put them back
    imputed_train.columns = train.columns
    imputed_test.columns = test.columns
    
    #restoring datatypes 
    imputed_train[numerical_cols] = imputed_train[numerical_cols].apply(pd.to_numeric, errors='coerce')
    imputed_test[numerical_cols] = imputed_test[numerical_cols].apply(pd.to_numeric, errors='coerce')
    print(imputed_train.shape,imputed_test.shape)
    return imputed_train,imputed_test 

def robustlabelencoder(train,test):
    from sklearn.preprocessing import LabelEncoder

    label_enc = LabelEncoderExt()
    cols = catVar2(train)
    print(cols)
    for col in cols:
        label_enc.fit(train[col])
        train[col] = label_enc.transform(train[col])
        test[col] = label_enc.transform(test[col])
        
    print(train.shape,test.shape)
    
    return train,test

def normalabelencoder(train,test,cols):
    from sklearn.preprocessing import LabelEncoder

    label_enc = LabelEncoder()
    
    print(cols)
    for col in cols:
        label_enc.fit(train[col])
        train[col] = label_enc.transform(train[col])
        test[col] = label_enc.transform(test[col])
    
    return train,test



def ohebygetdummis(train,test):
    pass

def onhotencoder(train,test):
    
    from sklearn.preprocessing import OneHotEncoder
    object_cols = catVar1(train) #catVar1 gives desired categorical column and not all object columns
    print(object_cols)
    
    OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
    OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(train[object_cols]))
    OH_cols_test = pd.DataFrame(OH_encoder.transform(test[object_cols]))
    
    column_name = OH_encoder.get_feature_names(object_cols)
    OH_cols_train.columns = column_name
    OH_cols_test.columns = column_name
    

    # One-hot encoding removed index; put it back
    OH_cols_train.index = train.index
    OH_cols_test.index = test.index

    # Remove desired categorical columns (will replace with one-hot encoding)
    num_train = train.drop(object_cols, axis=1)
    num_test = test.drop(object_cols, axis=1)

    # Add one-hot encoded columns to numerical/remaining features
    OH_train = pd.concat([num_train, OH_cols_train], axis=1)
    OH_test = pd.concat([num_test, OH_cols_test], axis=1)
    
    print(OH_train.shape,OH_test.shape)
    
    
    return OH_train,OH_test

def missingcheck(data):
    total = data.isnull().sum().sort_values(ascending=False)
    percent_1 = data.isnull().sum()/data.isnull().count()*100
    percent_2 = (np.round(percent_1, 1)).sort_values(ascending=False)
    missing_data = pd.concat([total, percent_2], axis=1, keys=['Total', '%']) #ptr
    return missing_data

def targetencoding(train,test,y_train):
    import category_encoders as ce
    # Create the encoder itself
    cat_features = catVar2(train)
    print(f'targest emcoding for features {cat_features}')
    target_enc = ce.TargetEncoder(cols=cat_features)

    

    # Fit the encoder using the categorical features and target
    target_enc.fit(train[cat_features], y_train)
    

    # Transform the features, rename the columns with _target suffix, and join to dataframe
    traintrgtenc = train.join(target_enc.transform(train[cat_features]).add_suffix('_target'))
    testtrgtenc = test.join(target_enc.transform(test[cat_features]).add_suffix('_target'))
    
    print(traintrgtenc.shape,testtrgtenc.shape)
    
    return traintrgtenc,testtrgtenc


def special(train,test):
    train['Geo_Loc'] = train['Geo_Loc'].str.replace(r'\D', '')
    train['Geo_Loc'] = pd.to_numeric(train['Geo_Loc'], errors='coerce') 
    
    test['Geo_Loc'] = test['Geo_Loc'].str.replace(r'\D', '')
    test['Geo_Loc'] = pd.to_numeric(test['Geo_Loc'], errors='coerce')
    
    train['Date'] = pd.to_datetime(train['Date'] ,errors='coerce')
    test['Date'] = pd.to_datetime(test['Date'] ,errors='coerce')
    
    train = train.assign(
               hour=train.Date.dt.hour,
               day=train.Date.dt.day,
               month=train.Date.dt.month,
               year=train.Date.dt.year
                        )
    
    test = test.assign(
               hour=test.Date.dt.hour,
               day=test.Date.dt.day,
               month=test.Date.dt.month,
               year=test.Date.dt.year
                        )
    print(train.shape,test.shape)
    
    return train, test

In [155]:
# def syntheticvariable(train , y ):
#     from imblearn.over_sampling import SMOTENC
#     from collections import Counter
#     categorical_features = [cname.index for cname in train.columns if train[cname].dtype == "object"]
#     out = np.argwhere(train.columns.isin(categorical_features)).ravel().tolist()
    
#     smote_nc = SMOTENC([3, 5, 6, 8, 10], random_state=0)  #instead of passing out i need to hardcode it.

#     X_resampled, y_resampled = smote_nc.fit_resample(train,y)
    
#     print(sorted(Counter(y_resampled).items()))
    
#     return X_resampled,y_resampled
    

In [156]:
#####crete new features###

In [157]:
def createIntercations(train,test,cat_features):
    import itertools
    print(f"Creating features on {cat_features}, with combination 2 for training data /n")
    interactionstrain = pd.DataFrame(index=train.index)
    
    for col1 ,col2 in  itertools.combinations(cat_features,2):   
        newcolname = col1 + "_" + col2 
        new_values = train[col1].map(str) + "_" + train[col2].map(str)
        interactionstrain[newcolname] = new_values

    
    train_df = train.join(interactionstrain)
    
    print(f"Creating features on {cat_features}, with combination 2 for testing data")
    interactionstest = pd.DataFrame(index=train.index)
    
    for col1 ,col2 in  itertools.combinations(cat_features,2):   
        newcolname = col1 + "_" + col2 
        new_values = test[col1].map(str) + "_" + test[col2].map(str)
        interactionstest[newcolname] = new_values

    test_df = test.join(interactionstest)
    
    print(train_df.shape,test_df.shape)
    
    
    return train_df,test_df


In [158]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

from sklearn.metrics import log_loss
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import classification_report
from sklearn.metrics import classification_report
    
# Function for comparing different approaches
def score_dataset(X_train, X_valid, y_train, y_valid):
    #model = RandomForestClassifier(random_state=0)
    #model.fit(X_train, y_train)
    from sklearn.tree import DecisionTreeClassifier

    from sklearn.ensemble import BaggingClassifier

    clf1 = DecisionTreeClassifier(max_features=45,max_depth=100,random_state=786)

    clf2 = BaggingClassifier(base_estimator=clf1, n_estimators=100, 
                             bootstrap=True,
                             bootstrap_features=False, oob_score=True, warm_start=False,
                             n_jobs=-1, random_state=786, verbose=0)
    clf2.fit(X_train,y_train)

    preds = clf2.predict(X_valid)
    target_names = ['class 0', 'class 1', 'class 2','class 3', 'class 4', 'class 5', 'class 6']
    print(classification_report(y_valid, preds, target_names=target_names,labels= [0,1,2,3,4,5,6]))
    return clf2.score(X_valid,y_valid) 

In [159]:
# # X_train, X_valid, y_train, y_valid
# clf1

In [160]:
X_trainS, X_validS = special(X_train ,X_valid)

/home/ryan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:143: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ryan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:144: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ryan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:146: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyd

(95837, 18) (51606, 18)


/home/ryan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:150: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [161]:
X_trainSDr, X_validSDr = dropcol(X_trainS,X_validS)

(95837, 17) (51606, 17)


In [162]:
X_trainSDrIM, X_validSDrIM = imputer(X_trainSDr,X_validSDr)

(95837, 17) (51606, 17)


In [1]:
missingcheck(X_trainSDrIM),missingcheck( X_validSDrIM) 

NameError: name 'missingcheck' is not defined

In [164]:
X_trainSDrIMinter1,X_validSDrIMinter1 = createIntercations(X_trainSDrIM, X_validSDrIM,cat_features = ['SectionViolations','RiskLevel','Reason'])

Creating features on ['SectionViolations', 'RiskLevel', 'Reason'], with combination 2 for training data /n
Creating features on ['SectionViolations', 'RiskLevel', 'Reason'], with combination 2 for testing data
(95837, 20) (51606, 20)


In [165]:
X_trainSDrIMinter2,X_validSDrIMinter2 = createIntercations(X_trainSDrIMinter1, X_validSDrIMinter1,cat_features = ['State','City','Street','LocationID','Geo_Loc']
)

Creating features on ['State', 'City', 'Street', 'LocationID', 'Geo_Loc'], with combination 2 for training data /n
Creating features on ['State', 'City', 'Street', 'LocationID', 'Geo_Loc'], with combination 2 for testing data
(95837, 30) (51606, 30)


In [166]:
# X_trainSDrIMintertgenc,X_validSDrIMinter2tgenc = targetencoding(X_trainSDrIMinter2,X_validSDrIMinter2,y_train)
X_trainSDrIMinter2

,LicenseNo,FacilityID,FacilityName,Type,Street,City,State,LocationID,Reason,SectionViolations,...,State_City,State_Street,State_LocationID,State_Geo_Loc,City_Street,City_LocationID,City_Geo_Loc,Street_LocationID,Street_Geo_Loc,LocationID_Geo_Loc
0,17200,16506,15739,RESTAURANT,8860,id-11235901,id_1890134,81902.0,COMPLAINT,3.0,...,id_1890134_id-11235901,id_1890134_8860,id_1890134_81902.0,id_1890134_12926,id-11235901_8860,id-11235901_81902.0,id-11235901_12926,8860_81902.0,8860_12926,81902.0_12926
1,28914,23558,22471,RESTAURANT,14044,id-11235901,id_1890134,81879.0,COMPLAINT,2.0,...,id_1890134_id-11235901,id_1890134_14044,id_1890134_81879.0,id_1890134_12319,id-11235901_14044,id-11235901_81879.0,id-11235901_12319,14044_81879.0,14044_12319,81879.0_12319
2,8996,12944,12287,RESTAURANT,6195,id-11235901,id_1890134,81892.0,CANVASS,32.0,...,id_1890134_id-11235901,id_1890134_6195,id_1890134_81892.0,id_1890134_11843,id-11235901_6195,id-11235901_81892.0,id-11235901_11843,6195_81892.0,6195_11843,81892.0_11843
3,6065,22133,21156,RESTAURANT,12468,id-11235901,id_1890134,81856.0,SHORT FORM COMPLAINT,34.0,...,id_1890134_id-11235901,id_1890134_12468,id_1890134_81856.0,id_1890134_8992,id-11235901_12468,id-11235901_81856.0,id-11235901_8992,12468_81856.0,12468_8992,81856.0_8992
4,15226,15672,14956,BAR,3307,id-11235901,id_1890134,81867.0,LICENSE,32.0,...,id_1890134_id-11235901,id_1890134_3307,id_1890134_81867.0,id_1890134_10614,id-11235901_3307,id-11235901_81867.0,id-11235901_10614,3307_81867.0,3307_10614,81867.0_10614
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95832,10984,12219,11623,RESTAURANT,3887,id-11235901,id_1890134,81867.0,CANVASS,32.0,...,id_1890134_id-11235901,id_1890134_3887,id_1890134_81867.0,id_1890134_9498,id-11235901_3887,id-11235901_81867.0,id-11235901_9498,3887_81867.0,3887_9498,81867.0_9498
95833,23404,9401,8915,RESTAURANT,1173,id-11235901,id_1890134,81911.0,CANVASS,3.0,...,id_1890134_id-11235901,id_1890134_1173,id_1890134_81911.0,id_1890134_16551,id-11235901_1173,id-11235901_81911.0,id-11235901_16551,1173_81911.0,1173_16551,81911.0_16551
95834,17061,7663,7284,RESTAURANT,7518,id-11235901,id_1890134,81870.0,COMPLAINT RE-INSPECTION,32.0,...,id_1890134_id-11235901,id_1890134_7518,id_1890134_81870.0,id_1890134_14718,id-11235901_7518,id-11235901_81870.0,id-11235901_14718,7518_81870.0,7518_14718,81870.0_14718
95835,4757,6345,6079,RESTAURANT,13253,id-11235901,id_1890134,81886.0,CANVASS RE-INSPECTION,32.0,...,id_1890134_id-11235901,id_1890134_13253,id_1890134_81886.0,id_1890134_12351,id-11235901_13253,id-11235901_81886.0,id-11235901_12351,13253_81886.0,13253_12351,81886.0_12351


In [168]:
X_trainSDrIMinter2lbl,X_validSDrIMinter2lbl = robustlabelencoder(X_trainSDrIMinter2,X_validSDrIMinter2)

['Type', 'Reason', 'SectionViolations_RiskLevel', 'SectionViolations_Reason', 'RiskLevel_Reason', 'State_Street', 'State_LocationID', 'State_Geo_Loc', 'City_Street', 'City_LocationID', 'City_Geo_Loc', 'Street_LocationID', 'Street_Geo_Loc', 'LocationID_Geo_Loc']
(95837, 30) (51606, 30)


In [169]:
X_trainSDrIMinter2lbl.head()

,LicenseNo,FacilityID,FacilityName,Type,Street,City,State,LocationID,Reason,SectionViolations,...,State_City,State_Street,State_LocationID,State_Geo_Loc,City_Street,City_LocationID,City_Geo_Loc,Street_LocationID,Street_Geo_Loc,LocationID_Geo_Loc
0,17200,16506,15739,301,8860,id-11235901,id_1890134,81902.0,2,3.0,...,id_1890134_id-11235901,15134,83,2987,15078,54,2985,15176,15145,14665
1,28914,23558,22471,301,14044,id-11235901,id_1890134,81879.0,2,2.0,...,id_1890134_id-11235901,4056,63,2368,4049,34,2366,4065,4055,9449
2,8996,12944,12287,301,6195,id-11235901,id_1890134,81892.0,0,32.0,...,id_1890134_id-11235901,12460,75,1896,12413,46,1894,12496,12471,12990
3,6065,22133,21156,301,12468,id-11235901,id_1890134,81856.0,11,34.0,...,id_1890134_id-11235901,2469,41,14436,2464,12,14423,2472,2465,2552
4,15226,15672,14956,35,3307,id-11235901,id_1890134,81867.0,6,32.0,...,id_1890134_id-11235901,9493,52,635,9460,23,635,9522,9502,6147


In [170]:
X_trainSDrIMinter2lblohe,X_validSDrIMinter2lblohe = onhotencoder(X_trainSDrIMinter2lbl,X_validSDrIMinter2lbl)

['City', 'State', 'RiskLevel', 'State_City']
(95837, 38) (51606, 38)


In [27]:
# #X_trainSDrIMinter2lblohe.head(2)
# X_trainSDrIMinter2lbloheqq,X_validSDrIMinter2lbloheqq = imputer(X_trainSDrIMinter2lbl,X_validSDrIMinter2lbl)

(95837, 48) (51606, 48)


In [28]:
# X_trainSDrIMinter2lbloheqq.shape

(95837, 48)

In [171]:
y_train.shape

(95837,)

In [172]:
X_validSDrIMinter2lblohe.shape

(51606, 38)

In [173]:
y_valid.shape

(51606,)

In [174]:
missingcheck(X_trainSDrIMinter2lblohe)

,Total,%
State_City_id_1890135_id-11275913,0,0.0
hour,0,0.0
RiskLevel_Reason,0,0.0
SectionViolations_Reason,0,0.0
SectionViolations_RiskLevel,0,0.0
logID,0,0.0
year,0,0.0
month,0,0.0
day,0,0.0
Geo_Loc,0,0.0


In [175]:
X_trainSDrIMinter2lblohe.columns

Index(['LicenseNo', 'FacilityID', 'FacilityName', 'Type', 'Street',
       'LocationID', 'Reason', 'SectionViolations', 'Geo_Loc', 'hour', 'day',
       'month', 'year', 'logID', 'SectionViolations_RiskLevel',
       'SectionViolations_Reason', 'RiskLevel_Reason', 'State_Street',
       'State_LocationID', 'State_Geo_Loc', 'City_Street', 'City_LocationID',
       'City_Geo_Loc', 'Street_LocationID', 'Street_Geo_Loc',
       'LocationID_Geo_Loc', 'City_id-11235901', 'City_id-11275913',
       'State_id_1890134', 'State_id_1890135', 'RiskLevel_High',
       'RiskLevel_Low', 'RiskLevel_Medium', 'RiskLevel_Uncertain',
       'State_City_id_1890134_id-11235901',
       'State_City_id_1890134_id-11275913',
       'State_City_id_1890135_id-11235901',
       'State_City_id_1890135_id-11275913'],
      dtype='object')

In [34]:
#cols_to_drop = []

# dtrain = X_trainSDrIMinter2lbloheqq.drop(cols_to_drop,axis =1)
# dtest = X_validSDrIMinter2lbloheqq.drop(cols_to_drop,axis =1)
    

In [35]:
# from sklearn.linear_model import LogisticRegression
# from sklearn.ensemble import ExtraTreesClassifier

# from sklearn.feature_selection import SelectFromModel
# extclf = ExtraTreesClassifier(n_estimators=100, random_state=7)

# # Set the regularization parameter C=1
# logistic = RandomForestClassifier(n_estimators=100, random_state=7 ).fit(X_trainSDrIMinter2lbloheqq, y_train)
# model = SelectFromModel(logistic, prefit=True)


# X_new = model.transform(X_trainSDrIMinter2lbloheqq)
# X_new

In [36]:
# selected_features = pd.DataFrame(model.inverse_transform(X_new), 
#                                  index=X_trainSDrIMinter2lbloheqq.index,
#                                  columns=X_trainSDrIMinter2lbloheqq.columns)

# # Dropped columns have values of all 0s, keep other columns 
# selected_columns = selected_features.columns[selected_features.var() != 0]

In [37]:
# selected_columns

In [38]:
# selected_columns

In [39]:
# from sklearn.feature_selection import SelectKBest, f_classif
# selector = SelectKBest(f_classif, k=20)

# X_new1 = selector.fit_transform(X_trainSDrIMinter2lbloheqq, y_train)
# X_new1


In [40]:
# selected_features1 = pd.DataFrame(selector.inverse_transform(X_new1), 
#                                  index=X_trainSDrIMinter2lbloheqq.index,
#                                  columns=X_trainSDrIMinter2lbloheqq.columns)

# # Dropped columns have values of all 0s, keep other columns 
# selected_columns1 = selected_features1.columns[selected_features1.var() != 0]

In [41]:
# selected_columns1

In [42]:
score_dataset(X_trainSDrIMinter2lbloheqq,X_validSDrIMinter2lbloheqq,y_train,y_valid)

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

     class 0       0.00      0.00      0.00        17
     class 1       0.79      0.71      0.75     10009
     class 2       0.48      0.38      0.43       465
     class 3       0.45      0.26      0.33      1641
     class 4       0.89      0.94      0.91     27842
     class 5       0.71      0.75      0.73      7138
     class 6       0.67      0.66      0.66      4494

    accuracy                           0.81     51606
   macro avg       0.57      0.53      0.54     51606
weighted avg       0.81      0.81      0.81     51606



0.8146533348835406

In [96]:
X_trainSDrIMLbl, X_validSDrIMLbl = lblencdr(X_trainSDrIM, X_validSDrIM)

['Type', 'Reason']


In [97]:
X_trainSDrIMLblOhe, X_validSDrIMLblOhe = onhotencoder(X_trainSDrIMLbl, X_validSDrIMLbl)

['City', 'State', 'RiskLevel']


In [98]:
score_dataset(X_trainSDrIMLblOhe,X_validSDrIMLblOhe,y_train,y_valid)

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

     class 0       0.00      0.00      0.00        17
     class 1       0.79      0.70      0.74     10009
     class 2       0.56      0.30      0.39       465
     class 3       0.49      0.18      0.26      1641
     class 4       0.87      0.96      0.91     27842
     class 5       0.72      0.73      0.72      7138
     class 6       0.71      0.60      0.65      4494

    accuracy                           0.81     51606
   macro avg       0.59      0.50      0.53     51606
weighted avg       0.80      0.81      0.80     51606



0.813839476029919